In [40]:
# @title
##%%
import torch
import torchvision
from torchvision import transforms, datasets
from torch import nn, optim
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

# Setup
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')

# Hyperparameters
batch_size = 64
epochs = 150
learning_rate1 = 0.0002
learning_rate2 = 0.004
noise_dim = 100

# import random
# torch.manual_seed(420)




Using device: cuda:0


In [ ]:
# augmentation (run once)

from PIL import Image
import os

input_dir = '/content/sample_data/Orange/sub'
output_dir = '/content/sample_data/Orange2/sub'
os.makedirs(output_dir, exist_ok=True)

# Define individual augmentations
augmentations = {
    'resize': transforms.Resize((128, 128)),
    'flip': transforms.Compose([
        transforms.Resize((128, 128)),
        transforms.RandomHorizontalFlip(p=1.0)
    ]),
    'rotate': transforms.Compose([
        transforms.Resize((128, 128)),
        transforms.RandomRotation(15)
    ]),
    'color': transforms.Compose([
        transforms.Resize((128, 128)),
       transforms.RandomVerticalFlip(p=1.0)
    ]),
    'affine': transforms.Compose([
        transforms.Resize((128, 128)),
        transforms.RandomAffine(degrees=0, translate=(0.1, 0.1), scale=(0.9, 1.1))
    ])
}

image_files = [f for f in os.listdir(input_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

for fname in image_files:
    img_path = os.path.join(input_dir, fname)
    img = Image.open(img_path).convert('RGB')

    for key, aug in augmentations.items():
        aug_img = aug(img)
        aug_img.save(os.path.join(output_dir, f"{os.path.splitext(fname)[0]}_{key}.jpg"))


In [25]:
#Load all data(run once)
IMAGE_SIZE = 128
CHANNELS = 3

import subprocess
subprocess.run(["ls", "-a"])
subprocess.run(["rm", "-r", "/content/sample_data/Orange2/.ipynb_checkpoints"])

transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1), scale=(0.9, 1.1)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])


data_dir = '/content/sample_data/Orange2' 
# Load dataset
dataset = datasets.ImageFolder(root=data_dir, transform=transform)

# DataLoader
train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)




In [42]:
# Vanilla model
import torch
import torch.nn as nn

# Generator
class Generator(nn.Module):
    def __init__(self, latent_dim=100, feature_maps=64, channels=3):
        super(Generator, self).__init__()
        self.model = nn.Sequential(

            nn.Linear(latent_dim, feature_maps * 8 * 4 * 4),

            nn.ReLU(True),
            nn.Unflatten(1, (feature_maps * 8, 4, 4)),
            nn.ConvTranspose2d(feature_maps * 8, feature_maps * 4, 4, 2, 1),

            nn.BatchNorm2d(feature_maps * 4),
            nn.ReLU(True),
            nn.ConvTranspose2d(feature_maps * 4, feature_maps * 2, 4, 2, 1),

            nn.BatchNorm2d(feature_maps * 2),
            nn.ReLU(True),
            nn.ConvTranspose2d(feature_maps * 2, feature_maps, 4, 2, 1),

            nn.BatchNorm2d(feature_maps),
            nn.ReLU(True),

            nn.ConvTranspose2d(feature_maps, channels, 4, 2, 1),
            nn.Tanh()
        )

    def forward(self, z):
        return self.model(z)

class Discriminator(nn.Module):
    def __init__(self, feature_maps=64, channels=3):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(channels, feature_maps, 4, 2, 1),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(feature_maps, feature_maps * 2, 4, 2, 1), 
            nn.BatchNorm2d(feature_maps * 2),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(feature_maps * 2, feature_maps * 4, 4, 2, 1),  
            nn.BatchNorm2d(feature_maps * 4),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(feature_maps * 4, feature_maps * 8, 4, 2, 1),  
            nn.BatchNorm2d(feature_maps * 8),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(feature_maps * 8, 1, 4, 1, 0),             
            nn.Sigmoid()
        )

    def forward(self, x):
        out = self.model(x)  
        return out.mean([2, 3]).view(-1, 1)


generator = Generator().to(device)
discriminator = Discriminator().to(device)

# Optimizers
optimizer_G = optim.Adam(generator.parameters(), lr=learning_rate1)
optimizer_D = optim.Adam(discriminator.parameters(), lr=learning_rate2)

# Loss function
criterion = nn.BCELoss()
g_losses = []
d_losses = []
# Training
for epoch in range(epochs):
  for i, (images, _) in enumerate(train_loader):
        real_images = images.to(device)
        real_labels = torch.ones(images.size(0), 1, device=device)
        fake_labels = torch.zeros(images.size(0), 1, device=device)


        optimizer_D.zero_grad()
        outputs = discriminator(real_images)
        d_loss_real = criterion(outputs, real_labels)

        noise = torch.randn(images.size(0), noise_dim, device=device)
        fake_images = generator(noise)
        outputs = discriminator(fake_images.detach())
        d_loss_fake = criterion(outputs, fake_labels)

        d_loss = d_loss_real + d_loss_fake
        d_loss.backward()
        optimizer_D.step()

        optimizer_G.zero_grad()
        outputs = discriminator(fake_images)
        g_loss = criterion(outputs, real_labels)
        g_loss.backward()
        optimizer_G.step()
  g_losses.append(g_loss.item())
  d_losses.append(d_loss.item())

  if (epoch+1) % 1 == 0: 
        print(f'Epoch [{epoch+1}/{epochs}], Batch [{i+1}/{len(train_loader)}], D Loss: {d_loss.item():.4f}, G Loss: {g_loss.item():.4f}')


# Save models
torch.save(generator.state_dict(), 'Vgenerator.pth')
torch.save(discriminator.state_dict(), 'Vdiscriminator.pth')



Epoch [1/25], Batch [1/1], D Loss: 1.3643, G Loss: 5.0689
Epoch [2/25], Batch [1/1], D Loss: 1.0695, G Loss: 6.1785
Epoch [3/25], Batch [1/1], D Loss: 0.4430, G Loss: 7.4033
Epoch [4/25], Batch [1/1], D Loss: 0.5335, G Loss: 6.8024
Epoch [5/25], Batch [1/1], D Loss: 0.9647, G Loss: 7.3593
Epoch [6/25], Batch [1/1], D Loss: 0.4257, G Loss: 9.4603
Epoch [7/25], Batch [1/1], D Loss: 0.1512, G Loss: 10.0640
Epoch [8/25], Batch [1/1], D Loss: 0.0545, G Loss: 9.4360
Epoch [9/25], Batch [1/1], D Loss: 0.3931, G Loss: 11.5634
Epoch [10/25], Batch [1/1], D Loss: 0.0460, G Loss: 12.3992
Epoch [11/25], Batch [1/1], D Loss: 0.0484, G Loss: 12.1160
Epoch [12/25], Batch [1/1], D Loss: 0.2441, G Loss: 9.9569
Epoch [13/25], Batch [1/1], D Loss: 0.9752, G Loss: 13.2102
Epoch [14/25], Batch [1/1], D Loss: 0.1124, G Loss: 14.9722
Epoch [15/25], Batch [1/1], D Loss: 0.2780, G Loss: 15.7592
Epoch [16/25], Batch [1/1], D Loss: 0.1681, G Loss: 15.7545
Epoch [17/25], Batch [1/1], D Loss: 0.0274, G Loss: 15.00

In [ ]:
# FastGAN
import torch
import torch.nn as nn

# Generator
class FastGANGenerator(nn.Module):
    def __init__(self, z_dim=100, base_channels=64):
        super(FastGANGenerator, self).__init__()
        self.init_size = 4
        self.l1 = nn.Linear(z_dim, base_channels * 8 * 4 * 4)

        self.upsample_blocks = nn.Sequential(
            nn.BatchNorm2d(base_channels * 8),
            nn.ReLU(),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(base_channels * 8, base_channels * 4, 3, 1, 1),
            nn.BatchNorm2d(base_channels * 4),
            nn.ReLU(),

            nn.Upsample(scale_factor=2),
            nn.Conv2d(base_channels * 4, base_channels * 2, 3, 1, 1),
            nn.BatchNorm2d(base_channels * 2),
            nn.ReLU(),

            nn.Upsample(scale_factor=2),
            nn.Conv2d(base_channels * 2, base_channels, 3, 1, 1),
            nn.BatchNorm2d(base_channels),
            nn.ReLU(),

            nn.Upsample(scale_factor=2),
            nn.Conv2d(base_channels, 3, 3, 1, 1),
            nn.Tanh()
        )

    def forward(self, z):
        out = self.l1(z).view(z.size(0), -1, self.init_size, self.init_size)
        return self.upsample_blocks(out)


# Discriminator
class FastGANDiscriminator(nn.Module):
    def __init__(self, base_channels=64):
        super(FastGANDiscriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(3, base_channels, 4, stride=2, padding=1),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(base_channels, base_channels * 2, 4, stride=2, padding=1),
            nn.BatchNorm2d(base_channels * 2),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(base_channels * 2, base_channels * 4, 4, stride=2, padding=1),
            nn.BatchNorm2d(base_channels * 4),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(base_channels * 4, 1, 4),
            nn.Sigmoid()
        )

    def forward(self, img):
      out = self.model(img) 
      return out.mean([2, 3]).view(-1, 1) 




generator = FastGANGenerator().to(device)
discriminator = FastGANDiscriminator().to(device)

# Optimizers
optimizer_G = optim.Adam(generator.parameters(), lr=learning_rate1)
optimizer_D = optim.Adam(discriminator.parameters(), lr=learning_rate2)

# Loss function
criterion = nn.BCELoss()
g_losses = []
d_losses = []
# Training
for epoch in range(epochs):
  for i, (images, _) in enumerate(train_loader):
        real_images = images.to(device)
        real_labels = torch.ones(images.size(0), 1, device=device)
        fake_labels = torch.zeros(images.size(0), 1, device=device)

        optimizer_D.zero_grad()
        outputs = discriminator(real_images)
        d_loss_real = criterion(outputs, real_labels)

        noise = torch.randn(images.size(0), noise_dim, device=device)
        fake_images = generator(noise)
        outputs = discriminator(fake_images.detach())
        d_loss_fake = criterion(outputs, fake_labels)

        d_loss = d_loss_real + d_loss_fake
        d_loss.backward()
        optimizer_D.step()

        optimizer_G.zero_grad()
        outputs = discriminator(fake_images)
        g_loss = criterion(outputs, real_labels)
        g_loss.backward()
        optimizer_G.step()
  g_losses.append(g_loss.item())
  d_losses.append(d_loss.item())

  if (epoch+1) % 1 == 0:  
        print(f'Epoch [{epoch+1}/{epochs}], Batch [{i+1}/{len(train_loader)}], D Loss: {d_loss.item():.4f}, G Loss: {g_loss.item():.4f}')


# Save models
torch.save(generator.state_dict(), 'generator.pth')
torch.save(discriminator.state_dict(), 'discriminator.pth')

##%%


Epoch [1/150], Batch [2/2], D Loss: 2.0058, G Loss: 1.9846
Epoch [2/150], Batch [2/2], D Loss: 1.2629, G Loss: 1.2797
Epoch [3/150], Batch [2/2], D Loss: 0.9265, G Loss: 1.6693
Epoch [4/150], Batch [2/2], D Loss: 1.4152, G Loss: 0.8537
Epoch [5/150], Batch [2/2], D Loss: 1.8369, G Loss: 1.0045
Epoch [6/150], Batch [2/2], D Loss: 1.6044, G Loss: 1.3242
Epoch [7/150], Batch [2/2], D Loss: 1.4978, G Loss: 1.3415
Epoch [8/150], Batch [2/2], D Loss: 1.2188, G Loss: 1.5669
Epoch [9/150], Batch [2/2], D Loss: 1.1973, G Loss: 1.3606
Epoch [10/150], Batch [2/2], D Loss: 1.2199, G Loss: 1.1647
Epoch [11/150], Batch [2/2], D Loss: 1.0928, G Loss: 1.1026
Epoch [12/150], Batch [2/2], D Loss: 0.9621, G Loss: 1.1065
Epoch [13/150], Batch [2/2], D Loss: 0.8848, G Loss: 1.1755
Epoch [14/150], Batch [2/2], D Loss: 0.7405, G Loss: 1.3248
Epoch [15/150], Batch [2/2], D Loss: 0.7953, G Loss: 1.3476
Epoch [16/150], Batch [2/2], D Loss: 0.8604, G Loss: 1.3012
Epoch [17/150], Batch [2/2], D Loss: 0.9669, G Lo

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import torch

# Generate image
with torch.no_grad():
    z = torch.randn(1, noise_dim).to(device)
    generated = generator(z).detach().cpu()  # shape: (1, C, H, W)

# Denormalize from [-1, 1] to [0, 1]
generated = (generated + 1) / 2

# Convert to NumPy image
img = generated.squeeze().numpy()  # shape: (C, H, W) or (H, W)

# Plot
if img.ndim == 3:  # RGB
    img = np.transpose(img, (1, 2, 0))  # (H, W, C)

plt.imshow(img, cmap='gray' if img.ndim == 2 else None)
plt.axis('off')
plt.show()


In [ ]:
#loss plot
print(g_losses)
epoch = range(1, len(g_losses) + 1)
plt.figure(figsize=(8, 5))
plt.plot(epoch, g_losses, label='Generator Loss')
plt.plot(epoch, d_losses, label='Discriminator Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('GAN Loss Over Epochs')
plt.legend()
plt.grid(True)
plt.show()
